# Лабораторная работа №1

## Задание:

С помощью метода get_prepared_all_vacancies_from_hh
загрузить 10000 вакансии с сайта hh.ru.

Договоритесь чтобы вакансии не повторялись.

Сохранить результат в виде csv файла с тремя столбцами:
1. id вакансии
2. сырой json
3. подготовленное описание вакансии

Не забывайте про обработку ошибок

## Подключение библиотек

In [1]:
import nltk                         # Для токенезации и лемматизации

from urllib.request import urlopen  # Пакет для чтения URL
from urllib.error import HTTPError  # Обработка ошибки HTTPError

import json                         # Работа с форматом .json

import re                           # Регулярные выражения

import pymystem3                    # Лемматизация для русского текста
import pymorphy2                    # Лемматизация для русского текста

from string import punctuation      # знаки препинаний

import concurrent.futures           # ThreadPoolExecutor

import pandas as pd                 # Для работы экспорта в формат .csv

import numpy as np                  # Для создания ndarray

nltk.download('punkt')              # Модель токенайзера из NLTK

nltk.download('stopwords')          # Набор стоп-слов

[nltk_data] Downloading package punkt to C:\Users\Sergey
[nltk_data]     Mikheev\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Sergey
[nltk_data]     Mikheev\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Чтение json с hh.ru

In [2]:
def download_json(url):
  try:
    response = urlopen(url, timeout=60)
    data_json = json.loads(response.read())
    return data_json
  except HTTPError as e:
    if e.code == 404:
      return {'description': '404 Page Not Found'}
    else:
      raise e
  
#пример: download_json('https://api.hh.ru/vacancies/43551857')

Проверка работы метода

In [3]:
this = download_json('https://api.hh.ru/vacancies/10000007')
print(json.dumps(this, sort_keys=True, indent=2, ensure_ascii=False))

{
  "accept_handicapped": false,
  "accept_incomplete_resumes": true,
  "accept_kids": false,
  "accept_temporary": false,
  "address": null,
  "allow_messages": true,
  "alternate_url": "https://hh.ru/vacancy/10000007",
  "apply_alternate_url": "https://hh.ru/applicant/vacancy_response?vacancyId=10000007",
  "archived": true,
  "area": {
    "id": "160",
    "name": "Алматы",
    "url": "https://api.hh.ru/areas/160"
  },
  "billing_type": {
    "id": "free",
    "name": "Бесплатная"
  },
  "branded_description": null,
  "code": null,
  "contacts": null,
  "created_at": "2014-03-17T15:09:31+0400",
  "department": null,
  "description": "<p><strong>Требование:</strong></p> <ul> <li>грамотная речь</li> <li>пользователь ПК</li> </ul> <p><strong>Обязанности:</strong></p> <ul> <li>обработка звонков</li> <li>консультация клиентов по телефону</li> <li>назначение встреч</li> </ul> <p><strong>Условия:</strong></p> <ul> <li>оклад + бонусы</li> <li>график работы с 10.00 до 19.00</li> <li>6\\1 </l

## Очистка предложения от HTML-тегов и других лишних символов

In [4]:
'''
РЕАЛИЗОВАТЬ remove_garbage
Используйте regex
IN: str '<p>ТРЕБОВАНИЯ: <br />- обязательно высшее или незаконченное высшее экономическое образование;...'
OUT: str 'ТРЕБОВАНИЯ: - обязательно высшее или незаконченное высшее экономическое образование;...'
'''

def remove_garbage(raw_text):
  # tags = r'<\S*\s*\S*>'
  tags = r'<.*?>'
  quot = r'&quot|«|»'
  puncts = r'•|∙|●'
  dash = r'–|—|―|–|--|_'
  zero_witdth = r'\u200b|\u200d'
  emoji = r'✔|☎️|❗️|⏰|❕|☎|▀|▄|★|⚡|▶|✔️|✂️|❶|❷|❸|❹|③|②|✅'
  banwords = [tags, quot, puncts, dash, zero_witdth, emoji]
  text = raw_text
  for ex in banwords:
    text = ' '.join(elem.strip() for elem in  re.split(ex, text))
  return text

Проверка работы метода

In [5]:
example = remove_garbage("<p>ТРЕБОВАНИЯ: <br />- обязательно высшее или незаконченное высшее экономическое образование;...")
print(example)
print("ТРЕБОВАНИЯ: - обязательно высшее или незаконченное высшее экономическое образование;...")

ТРЕБОВАНИЯ: - обязательно высшее или незаконченное высшее экономическое образование;...
ТРЕБОВАНИЯ: - обязательно высшее или незаконченное высшее экономическое образование;...


## Токенизация

In [6]:
'''
РЕАЛИЗОВАТЬ tokenize
Используйте word_tokenize из nltk.tokenize
IN: str 'ТРЕБОВАНИЯ: - обязательно высшее или незаконченное высшее экономическое образование;...'
OUT: list ['требования', ':', '-', 'обязательно', 'высшее', 'или', 'незаконченное', 'высшее', 'экономическое', 'образование']
'''

def tokenize(raw_text):
  tokens = nltk.word_tokenize(raw_text.lower(), language='russian')
  return tokens

Проверка работы метода

In [7]:
print(tokenize("ТРЕБОВАНИЯ: - обязательно высшее или незаконченное высшее экономическое образование;..."))
print(['требования', ':', '-', 'обязательно', 'высшее', 'или', 'незаконченное', 'высшее', 'экономическое', 'образование'])

['требования', ':', '-', 'обязательно', 'высшее', 'или', 'незаконченное', 'высшее', 'экономическое', 'образование', ';', '...']
['требования', ':', '-', 'обязательно', 'высшее', 'или', 'незаконченное', 'высшее', 'экономическое', 'образование']


## Приведение токенов к базовой форме

In [8]:
'''
РЕАЛИЗОВАТЬ to_base_form
Можно использовать и лемматизацию и стемминг: pymystem3, ntlk, Spacy, gensim
IN: str 'Красивая мама красиво мыла раму'
OUT: str 'красивый мама красиво мыть рама'
'''

## pymorphy2
# morph = pymorphy2.MorphAnalyzer(lang='ru')
# 
# def to_base_form(raw_text):
#   text = ''
#   for word in raw_text.split(' '):
#     text += ' ' + morph.parse(word)[0].normal_form
#   return text.strip()

## pymystem
def to_base_form(raw_text):
  lemmatizer = pymystem3.Mystem()
  tokens = lemmatizer.lemmatize(raw_text)
  return ''.join(elem for elem in tokens).strip()

Проверка работы метода

In [9]:
print(to_base_form('красивая мама красиво мыла раму'))
print('красивый мама красиво мыть рама')

красивый мама красиво мыть рама
красивый мама красиво мыть рама


## Удаление стоп-слов

In [10]:
'''
РЕАЛИЗОВАТЬ remove_stop_words
Используйте stopwords из nltk.corpus 
IN: list ['требования', ':', '-', 'обязательно', 'высший', 'или', 'незаконченный', 'высший', 'экономический', 'образование']
OUT: list ['требование', 'обязательно', 'высший', 'незаконченный', 'высший', 'экономический', 'образование']
'''

def remove_stop_words(raw_text):
  punc = [elem for elem in punctuation]
  banwords = nltk.corpus.stopwords.words(fileids='russian') + punc
  return ' '.join(elem.strip() for elem in raw_text if elem not in banwords).split()

Проверка работы метода

In [11]:
print(remove_stop_words(['требования', ':', '-', 'обязательно', 'высший', 'или', 'незаконченный', 'высший', 'экономический', 'образование']))
print(['требование', 'обязательно', 'высший', 'незаконченный', 'высший', 'экономический', 'образование'])

['требования', 'обязательно', 'высший', 'незаконченный', 'высший', 'экономический', 'образование']
['требование', 'обязательно', 'высший', 'незаконченный', 'высший', 'экономический', 'образование']


## Получение подготовленного описания вакансии 

In [12]:
'''
РЕАЛИЗОВАТЬ get_prepared_vacancy_description_from_hh
Используйте реализованные ранее методы
IN: int 72323
OUT1: str vacancy_json dump
OUT2: list[list] [['требование', 'обязательно', 'высший', 'незаконченный', 'высший', 'экономический', 'образование', ...], [...], ...]
'''
def get_prepared_vacancy_from_hh(vacancy_id):
  base_url = 'https://api.hh.ru/vacancies/'
  vacancy_json = download_json(base_url + str(vacancy_id))
  vacancy_description = vacancy_json['description']
  vacancy_description_sentences = nltk.tokenize.sent_tokenize(vacancy_description)
  prepared_vacancy_description = []
  for sentence in vacancy_description_sentences:
    ungarbaged = remove_garbage(sentence)
    based = to_base_form(ungarbaged)
    tokenized = tokenize(based)
    prepared = remove_stop_words(tokenized)
    prepared_vacancy_description += [prepared]
  return vacancy_json, prepared_vacancy_description

Проверка работы метода

In [13]:
prepared_json, prepared_desc = get_prepared_vacancy_from_hh(43551857)

In [14]:
print(prepared_json)
print('\n-----------------------\n')
print(prepared_desc)

{'id': '43551857', 'premium': False, 'billing_type': {'id': 'standard', 'name': 'Стандарт'}, 'relations': [], 'name': 'Рабочий в производственные цеха', 'insider_interview': None, 'response_letter_required': False, 'area': {'id': '68', 'name': 'Омск', 'url': 'https://api.hh.ru/areas/68'}, 'salary': {'from': 24000, 'to': 41000, 'currency': 'RUR', 'gross': False}, 'type': {'id': 'open', 'name': 'Открытая'}, 'address': {'city': 'Омск', 'street': 'улица Будеркина', 'building': '2', 'description': None, 'lat': 54.9591513865, 'lng': 73.437212985, 'raw': 'Омск, улица Будеркина, 2', 'metro': None, 'metro_stations': []}, 'allow_messages': True, 'site': {'id': 'hh', 'name': 'hh.ru'}, 'experience': {'id': 'noExperience', 'name': 'Нет опыта'}, 'schedule': {'id': 'shift', 'name': 'Сменный график'}, 'employment': {'id': 'full', 'name': 'Полная занятость'}, 'department': None, 'contacts': None, 'description': '<p><strong>Обязанности:</strong></p> <ul> <li>Работа в производственных цехах.</li> </ul> <

## Получение всех подготовленных вакансий

In [15]:
'''
РЕАЛИЗОВАТЬ get_prepared_all_vacancies_from_hh
Используйте get_prepared_vacancy_from_hh и ThreadPoolExecutor из concurrent.futures
IN: list [int]
OUT: list [vacancy_id, raw_description, prepared_description]
'''
##
# возвращаемое значение изменено с list[int, str, list[list[str]]] на list[list[int, str, list[list[str]]]]
##
def get_prepared_all_vacancies_from_hh(vacancy_ids):
  res = []
  with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = {executor.submit(get_prepared_vacancy_from_hh, vacancy_id): vacancy_id for vacancy_id in vacancy_ids}
    for future in concurrent.futures.as_completed(futures):
      json, desc = future.result()
      res.append([futures[future], json, desc])
  return res

Проверка работы метода

In [ ]:
id_list = np.random.randint(10000000, 45000000, 3500)
result = get_prepared_all_vacancies_from_hh(id_list)
# pymorphy2: 816.2s
# pymystem3: 3197.5s

In [19]:
csv = pd.DataFrame(result, columns=['id', 'raw_json', 'description'])
display(csv)

,id,raw_json,description
0,24231110,"{'id': '24231110', 'premium': False, 'billing_...","[[обязанность, выполнение, лабораторный, анали..."
1,12765148,"{'id': '12765148', 'premium': False, 'billing_...","[[требование, энергичность, сообразительность,..."
2,14242127,"{'id': '14242127', 'premium': False, 'billing_...","[[обязанность, прокладка, кабель, вдрс, устано..."
3,27756135,"{'id': '27756135', 'premium': False, 'billing_...","[[обязанность, устройство, стеновый, опалубка,..."
4,11108833,"{'id': '11108833', 'premium': False, 'billing_...","[[компания, тов, нвп, дніпро-мто, обязанность,..."
...,...,...,...
3495,24787031,"{'id': '24787031', 'premium': False, 'billing_...","[[ооо, пэк, многопрофильный, компания, осущест..."
3496,37818707,"{'id': '37818707', 'premium': False, 'billing_...","[[бояться, вызов, легко, находить, общий, язык..."
3497,32115756,"{'id': '32115756', 'premium': False, 'billing_...","[[компания, дикарт, завод, гипсовый, лепнина, ..."
3498,25642855,"{'id': '25642855', 'premium': False, 'billing_...","[[ооо, тк, гарда, самый, динамично, развивающи..."


Сохранение результата в .csv файл

In [21]:
csv.to_csv('csv/result.csv')

Проверка числа несуществующих вакансий

In [20]:
display(csv[csv.raw_json=="{'description': '404 Page Not Found'}"])

,id,raw_json,description
22,17699346,{'description': '404 Page Not Found'},"[['404', 'page', 'not', 'found']]"
44,37903819,{'description': '404 Page Not Found'},"[['404', 'page', 'not', 'found']]"
52,22905251,{'description': '404 Page Not Found'},"[['404', 'page', 'not', 'found']]"
56,21528255,{'description': '404 Page Not Found'},"[['404', 'page', 'not', 'found']]"
78,21377720,{'description': '404 Page Not Found'},"[['404', 'page', 'not', 'found']]"
...,...,...,...
3477,10852406,{'description': '404 Page Not Found'},"[['404', 'page', 'not', 'found']]"
3479,10064871,{'description': '404 Page Not Found'},"[['404', 'page', 'not', 'found']]"
3487,32508592,{'description': '404 Page Not Found'},"[['404', 'page', 'not', 'found']]"
3488,36759251,{'description': '404 Page Not Found'},"[['404', 'page', 'not', 'found']]"
